In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
interpreter = tf.lite.Interpreter(model_path='/content/drive/MyDrive/capstone/lite-model_movenet_singlepose_lightning_3.tflite')

In [ ]:
interpreter.allocate_tensors()
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 312,
  'shape': array([ 1,  1, 17,  3], dtype=int32),
  'shape_signature': array([ 1,  1, 17,  3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [ ]:
cap = cv2.VideoCapture("/content/drive/MyDrive/capstone/videos/Ahtesham Hussain crossing the road.mp4")

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
"""def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)"""

In [ ]:
"""def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) """

In [ ]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'}

In [ ]:
import tensorflow_hub as hub

In [ ]:
model_name = "movenet_thunder"  #["movenet_lightning", "movenet_thunder", "movenet_lightning.tflite", "movenet_thunder.tflite"]

if "tflite" in model_name:
  if "movenet_lightning" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/3?lite-format=tflite
    input_size = 192
  elif "movenet_thunder" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/3?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image"""
    # TF Lite format expects tensor type of float32.
    input_image = tf.cast(input_image, dtype=tf.float32)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/3")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/3")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image"""
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoint_with_scores = outputs['output_0'].numpy()
    return keypoint_with_scores

In [ ]:
cap = cv2.VideoCapture("/content/drive/MyDrive/capstone/single pose/Ahtesham Hussain crossing the road.mp4")

In [ ]:
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(total_frames)
frame=[]
for fno in range(0, total_frames,1):
  if(fno>40):
    break
  cap.set(cv2.CAP_PROP_POS_FRAMES, fno)
  _, image = cap.read()
  img = image.copy()
  img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256,256)
  input_image = tf.cast(img, dtype=tf.float32)
    
  # Setup input and output 
  #input_details = interpreter.get_input_details()
  #output_details = interpreter.get_output_details()
    
  # Make predictions 
  #keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
  #print(fno)
  keypoints_with_scores = movenet(input_image)
  k=keypoints_with_scores[0][0]
  #print(k)
  k1=k[:,0:2]
  #print(k1)
  k2=np.array(k1).flatten()
  #print(k2)
  k3=np.append(k2,1) 
  #print(k3)
  frame.append(k3)
cap.release()

324


In [ ]:
frame

[array([0.46175653, 0.62204134, 0.4559969 , 0.62574506, 0.45769304,
        0.62086731, 0.45557529, 0.63353044, 0.45673031, 0.62066984,
        0.48086205, 0.63764524, 0.48189297, 0.61884916, 0.51734322,
        0.63503289, 0.52166772, 0.61212283, 0.55034149, 0.62115937,
        0.54833072, 0.62401754, 0.54844844, 0.62521952, 0.54832911,
        0.62473142, 0.59238702, 0.61615962, 0.59144837, 0.63485903,
        0.63446665, 0.60398316, 0.63846779, 0.65487909, 1.        ]),
 array([0.45936432, 0.63312924, 0.45429146, 0.63559771, 0.45587939,
        0.63143706, 0.45572728, 0.63852066, 0.45621926, 0.62871921,
        0.47995037, 0.63924611, 0.48195127, 0.6228838 , 0.51248914,
        0.63671297, 0.51727796, 0.62055308, 0.53241664, 0.63626927,
        0.54306173, 0.6350655 , 0.54952508, 0.62831426, 0.54981697,
        0.62658417, 0.59199846, 0.62040615, 0.59423083, 0.63502192,
        0.63482434, 0.60716063, 0.63896406, 0.65546763, 1.        ]),
 array([0.46529993, 0.63977122, 0.46053183, 

In [ ]:
import pandas as pd 

In [ ]:
pd.DataFrame(frame)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.461757,0.622041,0.455997,0.625745,0.457693,0.620867,0.455575,0.633530,0.456730,0.620670,...,0.624731,0.592387,0.616160,0.591448,0.634859,0.634467,0.603983,0.638468,0.654879,1.0
1,0.459364,0.633129,0.454291,0.635598,0.455879,0.631437,0.455727,0.638521,0.456219,0.628719,...,0.626584,0.591998,0.620406,0.594231,0.635022,0.634824,0.607161,0.638964,0.655468,1.0
2,0.465300,0.639771,0.460532,0.643356,0.462195,0.636729,0.461058,0.645560,0.459532,0.630677,...,0.625944,0.596792,0.629066,0.592148,0.637110,0.632233,0.609641,0.635889,0.657868,1.0
3,0.464570,0.638495,0.459459,0.642993,0.459663,0.639049,0.459504,0.648027,0.458138,0.630231,...,0.623109,0.600331,0.634535,0.592999,0.639119,0.630280,0.617808,0.637774,0.662854,1.0
4,0.461593,0.637394,0.456162,0.642232,0.457225,0.637490,0.456917,0.649621,0.456770,0.629359,...,0.624126,0.600738,0.633482,0.592544,0.642353,0.630806,0.618496,0.637718,0.668646,1.0
5,0.465750,0.639405,0.460377,0.644498,0.460869,0.636583,0.461488,0.650999,0.459353,0.629828,...,0.620819,0.594326,0.631845,0.594511,0.638697,0.631804,0.619208,0.639227,0.666068,1.0
6,0.466309,0.640888,0.461361,0.645449,0.461527,0.638087,0.461483,0.652273,0.459311,0.631584,...,0.621191,0.596057,0.631384,0.596133,0.637876,0.632463,0.621485,0.638059,0.665406,1.0
7,0.471028,0.639174,0.465833,0.645805,0.465807,0.637207,0.465780,0.653699,0.465433,0.629679,...,0.621315,0.598542,0.630650,0.598725,0.636338,0.633290,0.624502,0.636143,0.657302,1.0
8,0.470302,0.642721,0.465824,0.647143,0.465794,0.639022,0.467034,0.653806,0.467529,0.632600,...,0.624070,0.596402,0.630893,0.595841,0.635614,0.634806,0.629784,0.635734,0.644982,1.0
9,0.467360,0.644972,0.463127,0.646986,0.463471,0.643085,0.464280,0.649500,0.462973,0.639477,...,0.631873,0.598241,0.629557,0.599764,0.637985,0.632729,0.632530,0.636014,0.647706,1.0


In [ ]:
def getpoints(path,mark):
  cap = cv2.VideoCapture(path)
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  print(total_frames)
  frame=[]
  for fno in range(0, total_frames,1):
    if(fno>100):
      break
    cap.set(cv2.CAP_PROP_POS_FRAMES, fno)
    _, image = cap.read()
    img = image.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 256,256)
    input_image = tf.cast(img, dtype=tf.float32)
      
    # Setup input and output 
    #input_details = interpreter.get_input_details()
    #output_details = interpreter.get_output_details()
      
    # Make predictions 
    #keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    #print(fno)
    keypoints_with_scores = movenet(input_image)
    k=keypoints_with_scores[0][0]
    #print(k)
    k1=k[:,0:2]
    #print(k1)
    k2=np.array(k1).flatten()
    #print(k2)
    if(fno<mark):
      k3=np.append(k2,0) 
    else:
      k3=np.append(k2,1) 
    #print(k3)
    frame.append(k3)
  cap.release()
  return frame


In [ ]:
def data(frame):
  print(frame)
  d=pd.DataFrame(frame)
  return d

In [ ]:
frame1=getpoints("/content/drive/MyDrive/capstone/single pose/madhurima di crossing the road.mp4",65)

335


In [ ]:
d1=data(frame1)

[array([0.45668021, 0.67353553, 0.44955087, 0.6782763 , 0.45092618,
       0.67055982, 0.45115986, 0.6917097 , 0.4489972 , 0.66978097,
       0.47090137, 0.69892645, 0.47326046, 0.65876764, 0.50552571,
       0.71530569, 0.50797182, 0.6538564 , 0.51704508, 0.70971251,
       0.52039224, 0.66484225, 0.53598595, 0.69245213, 0.5336203 ,
       0.66725731, 0.57970434, 0.69198704, 0.576801  , 0.67060047,
       0.61272562, 0.68785286, 0.60745084, 0.67394549, 0.        ]), array([0.46073017, 0.67794615, 0.45430613, 0.6810739 , 0.45444393,
       0.67413491, 0.454238  , 0.69482642, 0.4539403 , 0.66983724,
       0.47204885, 0.70130217, 0.47580865, 0.66054541, 0.51014024,
       0.71871418, 0.51416254, 0.65367043, 0.52148259, 0.7122919 ,
       0.52345526, 0.66480386, 0.53684378, 0.6948207 , 0.53485054,
       0.66866165, 0.5787257 , 0.69408977, 0.57637089, 0.66946137,
       0.61213589, 0.68960112, 0.60641384, 0.67357665, 0.        ]), array([0.46294087, 0.68090427, 0.45691514, 0.68336517, 0.

In [ ]:
d1

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.456680,0.673536,0.449551,0.678276,0.450926,0.670560,0.451160,0.691710,0.448997,0.669781,...,0.667257,0.579704,0.691987,0.576801,0.670600,0.612726,0.687853,0.607451,0.673945,0.0
1,0.460730,0.677946,0.454306,0.681074,0.454444,0.674135,0.454238,0.694826,0.453940,0.669837,...,0.668662,0.578726,0.694090,0.576371,0.669461,0.612136,0.689601,0.606414,0.673577,0.0
2,0.462941,0.680904,0.456915,0.683365,0.456142,0.676617,0.454007,0.695028,0.453745,0.671803,...,0.668333,0.579723,0.693409,0.577060,0.669596,0.613471,0.689684,0.608065,0.673591,0.0
3,0.465008,0.681036,0.459544,0.683598,0.458085,0.676691,0.459564,0.696146,0.457413,0.671425,...,0.671328,0.579517,0.693852,0.577638,0.671792,0.612159,0.688544,0.607506,0.674186,0.0
4,0.462724,0.679002,0.457043,0.681816,0.455874,0.674463,0.455769,0.694561,0.454451,0.668532,...,0.670119,0.579345,0.692392,0.577899,0.672080,0.611681,0.687927,0.606738,0.674863,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.437900,0.618553,0.431801,0.622148,0.433039,0.616600,0.431899,0.631373,0.433906,0.620435,...,0.626352,0.567785,0.645661,0.565755,0.624278,0.607799,0.660107,0.603348,0.634202,1.0
97,0.438139,0.619144,0.433298,0.619934,0.433180,0.617342,0.432363,0.631954,0.434040,0.620554,...,0.625818,0.565073,0.644896,0.563326,0.623684,0.604619,0.657389,0.599731,0.633709,1.0
98,0.437058,0.620652,0.433945,0.619919,0.433777,0.617411,0.432931,0.632052,0.435005,0.620236,...,0.625862,0.565012,0.645664,0.562844,0.622112,0.605047,0.658952,0.600518,0.631823,1.0
99,0.435697,0.621019,0.432405,0.620556,0.432093,0.617536,0.431473,0.632329,0.432924,0.620034,...,0.626413,0.566279,0.645716,0.561658,0.622999,0.604997,0.658692,0.599840,0.633812,1.0


In [ ]:
d1.tail(40)

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
61,0.442298,0.652849,0.436924,0.657470,0.436652,0.651367,0.437078,0.670240,0.437832,0.656380,...,0.657224,0.564661,0.676728,0.557717,0.651769,0.608984,0.679765,0.600277,0.658587,0.0
62,0.440625,0.656533,0.435982,0.655632,0.435896,0.653393,0.434539,0.670099,0.437490,0.658542,...,0.657515,0.569791,0.676327,0.558099,0.651243,0.613220,0.678076,0.602764,0.655988,0.0
63,0.436454,0.650888,0.432036,0.651897,0.431546,0.650416,0.431975,0.665497,0.432656,0.658131,...,0.658305,0.568522,0.675277,0.560453,0.649765,0.612597,0.677540,0.605241,0.655222,0.0
64,0.436409,0.649970,0.431839,0.651065,0.431520,0.649542,0.431692,0.664759,0.432543,0.657099,...,0.658826,0.571024,0.676767,0.563116,0.649972,0.613867,0.679558,0.606770,0.654055,0.0
65,0.433204,0.650091,0.428974,0.650707,0.429135,0.649376,0.429251,0.663782,0.430967,0.656726,...,0.656838,0.568511,0.677473,0.561462,0.650722,0.611773,0.680374,0.604696,0.651704,1.0
66,0.432276,0.650675,0.428419,0.651120,0.428060,0.649876,0.428421,0.664474,0.429704,0.657397,...,0.655099,0.571211,0.678149,0.565850,0.650335,0.613834,0.680184,0.608903,0.650325,1.0
67,0.435447,0.646982,0.429278,0.651597,0.430705,0.646180,0.430934,0.662557,0.432069,0.653244,...,0.655953,0.572584,0.679413,0.569269,0.650685,0.614845,0.680412,0.611336,0.649728,1.0
68,0.441427,0.650687,0.436759,0.651720,0.437864,0.649329,0.437758,0.663003,0.440271,0.654498,...,0.658536,0.574671,0.678276,0.568006,0.650178,0.615942,0.679511,0.608798,0.646684,1.0
69,0.439171,0.652008,0.434739,0.653775,0.436014,0.647088,0.435619,0.664164,0.437618,0.651426,...,0.653065,0.576176,0.677536,0.572016,0.648465,0.615491,0.680025,0.608124,0.645497,1.0
70,0.436707,0.649781,0.431904,0.651862,0.433289,0.645950,0.432166,0.663601,0.434017,0.653004,...,0.655803,0.575484,0.675461,0.569525,0.648599,0.615033,0.678274,0.605480,0.644154,1.0
